In [1]:
import pandas as pd
import os
import numpy as np
import copy
import random

In [67]:
def make_derangement(num_index, how_many):
    idx = np.arange(num_index)
    
    sf = copy.deepcopy(idx)
    results = []
    for _ in range(min(num_index-1, how_many)):
        # make a derangement
        while True:
            np.random.shuffle(sf)
            if not np.any((sf == idx)):
                results.append(sf.tolist())
                break
    return results

print(make_derangement(10, 5))
print()
print(make_derangement(3, 5))

[[1, 2, 5, 9, 8, 3, 7, 6, 0, 4], [3, 7, 9, 8, 5, 0, 4, 6, 2, 1], [3, 2, 5, 9, 1, 0, 8, 6, 7, 4], [8, 2, 4, 6, 0, 9, 3, 5, 7, 1], [6, 7, 1, 5, 9, 4, 8, 0, 3, 2]]

[[1, 2, 0], [1, 2, 0]]


In [5]:
no_new_whale_list = pd.read_csv('../../data/all/train_no_new_whale.csv')
new_whale_list = pd.read_csv('../../data/all/new_whale.csv')
print(no_new_whale_list.head())
print(new_whale_list.head())

           Image         Id
0  0000e88ab.jpg  w_f48451c
1  0001f9222.jpg  w_c3d896a
2  00029d126.jpg  w_20df2c5
3  000a6daec.jpg  w_dd88965
4  0016b897a.jpg  w_64404ac
           Image         Id
0  00050a15a.jpg  new_whale
1  0005c1ef8.jpg  new_whale
2  0006e997e.jpg  new_whale
3  000f0f2bf.jpg  new_whale
4  001d7450c.jpg  new_whale


In [93]:
no_new_whale_list_len = no_new_whale_list.shape[0]
print(no_new_whale_list_len)

15697


In [22]:
# get all ids for known whales
known_ids = no_new_whale_list.Id.unique().tolist()
print(known_ids[:10])
print(len(known_ids))

['w_f48451c', 'w_c3d896a', 'w_20df2c5', 'w_dd88965', 'w_64404ac', 'w_a6f9d33', 'w_581ba42', 'w_cb622a2', 'w_d3b46e7', 'w_8cad422']
5004


## make match pairs

In [80]:
# make match pairs, in form: im_a, im_b, id_a, id_b, label, label == 1 means they're
# from a same whale

match_pairs = pd.DataFrame(columns=['im_a', 'im_b', 'id_a', 'id_b', 'label'])

for id in known_ids:
    one_whale = no_new_whale_list[no_new_whale_list.Id == id]
    one_whale = one_whale.reset_index(drop=True)
    one_whale_len = one_whale.shape[0]
    
    if  one_whale_len == 1:  # if len(tmp_df) == 1, just cpoy this image for im_b
        im_a = one_whale.Image[0]
        im_b = one_whale.Image[0]
        id_a = id
        id_b = id
        label = int(1)
        match_pairs = match_pairs.append({'im_a':im_a, 'im_b':im_b, 'id_a':id_a,
                                          'id_b':id_b, 'label':label},
                                         ignore_index=True)
    else:
        derangements = make_derangement(one_whale_len, 5)
        for drg in derangements:
            one_drged = one_whale.iloc[drg, :]
            match_pairs = match_pairs.append(
                pd.DataFrame({'im_a':one_whale.Image.iloc[:].as_matrix().tolist(),
                 'im_b':one_drged.Image.iloc[:].as_matrix().tolist(), 
                 'id_a':id, 'id_b':id, 'label':int(1)}),
                ignore_index=True)


In [90]:
match_pairs.to_csv('../../data/all/match_pairs_5.csv', index=False)
match_pairs.shape

(51757, 5)

## make different pairs

sicne we made 51757 match_paris, we now will make similar num of diffrent paris, for 15697 images in no_new_whale_list, we make 4 pairs for each image, 2 from no_new_whale_list, another 2 from new_whale_list

In [101]:
different_pairs = pd.DataFrame(columns=['id_a', 'id_b', 'im_a', 'im_b', 'label'])

for i in range(no_new_whale_list_len):
    one_whale = no_new_whale_list.iloc[i,:]
    id_a = one_whale.Id
    im_a = one_whale.Image
    known_candidate = no_new_whale_list[no_new_whale_list.Id != id_a]
    idxs_known = np.random.choice(np.arange(known_candidate.shape[0]).tolist(), 2)
    idxs_new_whale = np.random.choice(np.arange(new_whale_list.shape[0]).tolist(), 2)
    concated = pd.concat([known_candidate.iloc[idxs_known, :],
                          new_whale_list.iloc[idxs_new_whale, :]],
                        ignore_index=True)
    different_pairs = different_pairs.append(
        pd.DataFrame({'id_a': id_a,
                      'id_b': concated.Id.iloc[:].as_matrix().tolist(),
                      'im_a': im_a,
                      'im_b': concated.Image.iloc[:].as_matrix().tolist(),
                      'label': int(0)}), 
        ignore_index=True)

print(different_pairs.shape)

(62788, 5)


In [102]:
print(different_pairs.iloc[200:400])
print(different_pairs.iloc[-400:-200])
different_pairs.to_csv('../../data/all/different_pairs_5.csv', index=False)

          id_a       id_b           im_a           im_b label
200  w_dd944b7  w_e06d673  00d641885.jpg  c25057c98.jpg     0
201  w_dd944b7  w_75b6e81  00d641885.jpg  86d03a392.jpg     0
202  w_dd944b7  new_whale  00d641885.jpg  fa73bc2ab.jpg     0
203  w_dd944b7  new_whale  00d641885.jpg  363890e2f.jpg     0
204  w_6f0cbe3  w_dc94d80  00d6cbcbf.jpg  1a58291b0.jpg     0
205  w_6f0cbe3  w_15b5eb1  00d6cbcbf.jpg  a5d9e12ce.jpg     0
206  w_6f0cbe3  new_whale  00d6cbcbf.jpg  8b174b2c4.jpg     0
207  w_6f0cbe3  new_whale  00d6cbcbf.jpg  f350d57ad.jpg     0
208  w_6cfa650  w_8a1786f  00d7dc401.jpg  a4c0b6772.jpg     0
209  w_6cfa650  w_b5e6c9c  00d7dc401.jpg  78006340a.jpg     0
210  w_6cfa650  new_whale  00d7dc401.jpg  f86689ed1.jpg     0
211  w_6cfa650  new_whale  00d7dc401.jpg  cdcacd8d0.jpg     0
212  w_6e209a8  w_ec8ac93  00d831b06.jpg  36fec5a40.jpg     0
213  w_6e209a8  w_6cda039  00d831b06.jpg  b57048db4.jpg     0
214  w_6e209a8  new_whale  00d831b06.jpg  710a05e3e.jpg     0
215  w_6

In [107]:
merged_list = pd.concat([match_pairs, different_pairs], ignore_index=True)
shuffle_idx = np.arange(merged_list.shape[0])
np.random.shuffle(shuffle_idx)
np.random.shuffle(shuffle_idx)
np.random.shuffle(shuffle_idx)
merged_list = merged_list.iloc[shuffle_idx,:]
merged_list = merged_list.reset_index(drop=True)
print(merged_list.shape)
merged_list.iloc[:100,:]

(114545, 5)


,id_a,id_b,im_a,im_b,label
0,w_302799d,w_fa955f1,040d790e5.jpg,d6f3cd0f6.jpg,0
1,w_7c60e77,w_7c60e77,31687c1c1.jpg,065b72556.jpg,1
2,w_700ebb4,w_700ebb4,a290b92a7.jpg,8c97c8b7c.jpg,1
3,w_c063530,w_3b03149,1eb6554f3.jpg,439320f0c.jpg,0
4,w_b71ac68,w_b71ac68,8e24cd62d.jpg,c2e1bf0c1.jpg,1
5,w_14e5fe2,w_14e5fe2,9ba1bc891.jpg,6130eb542.jpg,1
6,w_c5260b8,w_c5260b8,14b5efc26.jpg,efcbb3761.jpg,1
7,w_5a696c2,w_cd9dec7,ddfa288e4.jpg,a20b60095.jpg,0
8,w_04713fd,w_6e97ef4,2ba91c238.jpg,953768181.jpg,0
9,w_e2a09d4,w_3de579a,1e7271c12.jpg,538eb9102.jpg,0


In [108]:
merged_list.to_csv('../../data/all/siamese_pairs_5.csv', index=False)

### split siamese_pairs_5_all into train and val split

In [2]:
siamese_paris_5_all = pd.read_csv('../../data/all/siamese_pairs_5_all.csv')
siamese_paris_5_all.head()

,id_a,id_b,im_a,im_b,label
0,w_302799d,w_fa955f1,040d790e5.jpg,d6f3cd0f6.jpg,0
1,w_7c60e77,w_7c60e77,31687c1c1.jpg,065b72556.jpg,1
2,w_700ebb4,w_700ebb4,a290b92a7.jpg,8c97c8b7c.jpg,1
3,w_c063530,w_3b03149,1eb6554f3.jpg,439320f0c.jpg,0
4,w_b71ac68,w_b71ac68,8e24cd62d.jpg,c2e1bf0c1.jpg,1


In [6]:
all_idx = np.arange(siamese_paris_5_all.shape[0])
print(all_idx[:20])
np.random.shuffle(all_idx)
np.random.shuffle(all_idx)
np.random.shuffle(all_idx)
print(all_idx[:20])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[19583 34809 40938 10148 58967 40536 72198 45117 73642 98074 42995 24867
 30626 98984 91996 74876 36401 80298 63154 23813]


In [9]:
train_split_long = int(siamese_paris_5_all.shape[0] * 0.9)
train_split_idx = all_idx[:train_split_long]
val_split_idx = all_idx[train_split_long:]
print(train_split_idx.shape, train_split_idx, train_split_idx.min(), train_split_idx.max())
print(val_split_idx.shape, val_split_idx, val_split_idx.min(), val_split_idx.max())

(103090,) [ 19583  34809  40938 ... 108000  20986  35276] 0 114544
(11455,) [29144 93637 96980 ... 51562 14775 88387] 9 114537


In [10]:
train_split = siamese_paris_5_all.iloc[train_split_idx]
val_split = siamese_paris_5_all.iloc[val_split_idx]

In [12]:
print(train_split.shape)
train_split.head()

(103090, 5)


,id_a,id_b,im_a,im_b,label
19583,w_de657c1,w_de657c1,c3769dc73.jpg,6613985b1.jpg,1
34809,w_5a2634c,w_700ebb4,386098eae.jpg,f44c6c117.jpg,0
40938,w_99e43ae,w_99e43ae,de8d7aa2f.jpg,411ff2aeb.jpg,1
10148,w_1e85eea,new_whale,e8030f109.jpg,d7b3b1627.jpg,0
58967,w_c8d05b6,w_df32eec,701135c90.jpg,8159ad49a.jpg,0


In [13]:
print(val_split.shape)
val_split.head()

(11455, 5)


,id_a,id_b,im_a,im_b,label
29144,w_518a722,w_00a41ba,a519f5eef.jpg,847b16277.jpg,0
93637,w_5a2634c,w_5a2634c,3f7010135.jpg,814ebcb37.jpg,1
96980,w_fec5547,new_whale,6dc294615.jpg,f663f1060.jpg,0
14442,w_78ab4d4,w_78ab4d4,e6710386e.jpg,692c65701.jpg,1
72466,w_55a34c7,w_55a34c7,d63a01542.jpg,aeb320a85.jpg,1


In [14]:
train_split.to_csv('../../data/all/siamese_pairs_5_train.csv', index=False)
val_split.to_csv('../../data/all/siamese_pairs_5_val.csv', index=False)